# Run with Data Collection

This notebook combines data collection with the Run notebook. Any time the JetBot gets confused, save an image to enhance the dataset.

In [1]:
import torch
import torchvision
import cv2
import numpy as np
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot
import torch.nn.functional as F
import time
import os

# Restart camera service
os.system('systemctl restart nvargus-daemon')

# Alexnet model for CA
model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

# Resnet model for steering
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

# Load the trained weights model
model.load_state_dict(torch.load('best_model_FB.pth'))

device = torch.device('cuda')
model = model.to(device)


# Begin image preprocessing
mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x
# End image preprocessing

# Begin camera instantiation
camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
# End camera instantiation

# display the camera with sliders to show free/blocked
#display(widgets.HBox([image, blocked_slider]))

# Instantiate JetBot
from jetbot import Robot

robot = Robot()

# Begin update function to apply model to camera input
def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.5:
        robot.forward(0.4)
    else:
        robot.left(0.4)
    
    time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to intialize

In [3]:
# this attaches the 'update' function to the 'value' traitlet of our camera
camera.observe(update, names='value')  

Stop the JetBot:

In [4]:
import time

camera.unobserve(update, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()